In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import rp2
from rp2 import hagai_2018, ui, notebooks, regression

_ = notebooks.initialise_environment(
    "Mean_Variance_Fits",
    dependencies=["Data_Setup"],
)

In [ ]:
species_to_compare = ["mouse", "pig", "rabbit", "rat"]

In [ ]:
lps_responsive_mouse_genes = hagai_2018.load_lps_responsive_genes()
analysis_orthologues = rp2.load_mouse_orthologues().loc[lps_responsive_mouse_genes].reset_index()
analysis_orthologues.columns = analysis_orthologues.columns.str.slice(0, -5)

In [ ]:
gene_info_df = pd.concat([rp2.load_biomart_gene_symbols_df(species) for species in species_to_compare])

In [ ]:
def create_stats(species, count_type="median"):
    counts_adata = hagai_2018.load_counts(species, scaling=count_type)
    counts_adata = counts_adata[:, analysis_orthologues[species]].copy()
    stats = hagai_2018.calculate_counts_condition_stats(counts_adata)
    stats.insert(1, "species", species)
    return stats


stats_df = pd.concat([create_stats(species) for species in species_to_compare])

In [ ]:
mv_lr_df = stats_df.groupby("gene").apply(regression.fit_robust_linear_trend, x_var="mean", y_var="variance")

In [ ]:
@widgets.interact(mouse_gene_id=ui.make_gene_selector(gene_info_df.loc[analysis_orthologues.mouse].symbol, rows=5))
def plot_mv(mouse_gene_id):
    gene_ids = analysis_orthologues.loc[analysis_orthologues.mouse == mouse_gene_id].squeeze().to_list()
    stats_subset = stats_df.loc[stats_df.gene.isin(gene_ids)]
    lr_subset = mv_lr_df.loc[gene_ids]

    plot_df = stats_subset[["gene", "species", "replicate", "treatment", "time_point", "mean", "variance"]].copy()
    plot_df = plot_df.merge(lr_subset, on="gene").sort_values(by=["gene", "mean"])
    plot_df["lr_y"] = (plot_df["mean"] * plot_df["slope"]) + plot_df["intercept"]

    colours = sns.color_palette()[:len(gene_ids)]

    plot_output = widgets.Output()
    with plot_output:
        sns.scatterplot(
            plot_df["mean"],
            plot_df["variance"],
            style=plot_df["treatment"],
            hue=plot_df["species"],
            hue_order=species_to_compare,
            palette=colours,
        )

        ax = sns.lineplot(
            plot_df["mean"],
            plot_df["lr_y"],
            hue=plot_df["species"],
            hue_order=species_to_compare,
            palette=colours,
            legend=None,
        )
        for line in ax.lines:
            line.set_linestyle("--")

        x_max = plot_df["mean"].max()
        y_max = plot_df["variance"].max()

        plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
        plt.xlim(left=-(x_max / 20))
        plt.ylim(bottom=-(y_max / 20))
        plt.show()

    info_output = widgets.Output()
    with info_output:
        display(lr_subset)

    display(widgets.HBox([plot_output, info_output]))